# クロス集計とカイ二乗検定

- ``core.py``のモジュールに、クロス集計＆カイ二乗検定するメソッドを追加する
  - ``crosstab_data`` : クロス集計した結果をカイ二乗検定する
  - ``crosstab_heatmap`` : クロス集計した結果をヒートマップにする
  - ``crosstab``: 上記のメソッドをまとめて実行する

In [ ]:
import itertools
import altair as alt
import pandas as pd
import titanite as ti
import scipy as sp

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciPy {sp.__version__}")
print(f"Titanite {ti.__version__}")

データを読み込む

- 前処理済のデータなので``Data``クラスで読み込む
  - データ読み込みをシンプルにできた
  - 外から``Config``にアクセスできないデメリットがある

In [ ]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
data = d.read()
# data.info()

クロス集計すると、データのカテゴリ情報がなくなってしまうので、対処法を調べてみた

- ``pd.DataFrame.dtypes``ですべてのカラムの型を確認できる
- ``pd.Series.dtypes`` / ``pd.Series.dtype``でカラムの型を確認できる
- ``data.cat.categories``でカテゴリーの中身にアクセスできる
- 型の変換なので``pd.DataFrame.astype``でキャストすればよい
- ``astype``には``pd.Series.dtype``を渡せばよいことがわかった

In [ ]:
# data.dtypes

In [ ]:
# data["q01"].cat.categories

In [ ]:
# for h in data.columns:
#     t = data[h].dtype
#     if t == "category":
#         c = data[h].cat.categories
#         print(f"{h} = {c}")

クロス集計してカイ二乗検定する

In [ ]:
x = "q02"
y = "q03"
v = "count"

# data[h0].dtype
# data[h1].dtype

In [ ]:
cross_tab = pd.crosstab(data[x], data[y])
# cross_tab
# cross_tab.columns
# cross_tab.info()

クロス集計した結果をロングデータに変換する（グラフ作成のための準備）
- ``reset_index``するときに、カラムの型情報が抜けてしまった
- カラム名が同じなので、元データのカテゴリ型を使って、上書きする

In [ ]:
melted = cross_tab.reset_index().melt(
    id_vars=x,
    var_name=y,
    value_name=v,
)
# ti.categorical_data(melted, category)
melted[x] = melted[x].astype(data[x].dtype)
melted[y] = melted[y].astype(data[y].dtype)
# melted.info()

これまでの処理をまとめる

In [ ]:
from scipy.stats import chi2_contingency


def crosstab(data: pd.DataFrame, header: tuple):
    x = header[0]
    y = header[1]
    z = "count"

    # クロス集計してカイ二乗検定
    cross_tab = pd.crosstab(data[x], data[y])
    chi2_test = chi2_contingency(cross_tab)

    # ロングデータに変換
    melted = cross_tab.reset_index().melt(id_vars=x, var_name=y, value_name=z)
    # 元データのカテゴリ型情報を付け直す
    melted[x] = melted[x].astype(data[x].dtype)
    melted[y] = melted[y].astype(data[y].dtype)

    # グラフを作成
    base = alt.Chart(melted).encode(
        alt.X(x),
        alt.Y(y),
    )

    mark = base.mark_rect().encode(
        color=alt.condition(
            alt.datum[z] > 0,
            alt.Color(f"{z}:Q"),
            alt.value("white"),
        )
    )

    text = base.mark_text().encode(
        text=alt.condition(alt.datum[z] > 0, alt.Text(f"{z}:Q"), alt.value("")),
        color=alt.value("black"),
    )

    heatmap = (mark + text).properties(
        width=400,
        height=400,
    )
    return cross_tab, chi2_test, heatmap

In [ ]:
cross_tab, chi2_test, hm = crosstab(data, ("q01", "q02"))

In [ ]:
# cross_tab.info()
cross_tab

In [ ]:
hm

In [ ]:
print(chi2_test.statistic)
print(chi2_test.pvalue)
print(chi2_test.dof)

カイ二乗検定の結果をCSV（or JSON）に書き出す

- まずデータフレームに変換する
- JSONはTypstで読み込みやすい
- CSVはTypstでそのまま表にできそう

In [ ]:
headers = [
    ("q01", "q02"),
    ("q13_binned", "q14"),
    ("q03_subregional", "q19"),
]

In [ ]:
chi2_tests = []
for header in headers:
    cross_tab, chi2_test, hm = crosstab(data, header)
    # display(cross_tab)
    # hm.display()
    x, y = header
    name = f"{x}-{y}"
    print(name)
    print(chi2_test.statistic)
    print(chi2_test.pvalue)
    print(chi2_test.dof)
    r = [name, chi2_test.statistic, chi2_test.pvalue, chi2_test.dof]
    chi2_tests.append(r)

In [ ]:
chi2_tests

In [ ]:
pd.DataFrame(chi2_tests, columns=["name", "statistic", "p-value", "dof"])